In [1]:
from langgraph.graph import StateGraph, START , END
from langchain_core.prompts import PromptTemplate
from typing import TypedDict, Literal
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
model = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0
)

In [3]:
class Sentiment(BaseModel):
    sentiment: Literal['positive','negative']

class Diagnosis(BaseModel):
    issue_type: Literal["product_quality", "service", "delivery", "pricing", "support", "other"] = Field(
        ..., description="The main category of issue raised in the review."
    )
    tone: Literal["angry", "disappointed", "frustrated", "sarcastic", "neutral-negative"] = Field(
        ..., description="The emotional style of the review."
    )
    sentiment_strength: Literal["mild", "moderate", "strong", "extreme"] = Field(
        ..., description="The intensity of the negative sentiment."
    )
    specificity: Literal["vague", "somewhat_specific", "specific"] = Field(
        ..., description="How detailed and concrete the review is."
    )
    constructiveness: Literal["venting", "non_actionable", "actionable"] = Field(
        ..., description="Whether the review provides actionable feedback or just complaints."
    )
    authenticity: Literal["genuine", "spam", "troll", "biased"] = Field(
        ..., description="Whether the review appears authentic or suspicious."
    )
    politeness: Literal["polite", "neutral", "rude"] = Field(
        ..., description="The politeness or offensiveness of the review tone."
    )
    impact_severity: Literal["low", "medium", "high"] = Field(
        ..., description="How serious or impactful the issue raised in the review is."
    )
    improvement_suggestion: Literal["explicit", "implicit", "none"] = Field(
        ..., description="Whether the review suggests improvements explicitly, implicitly, or not at all."
    )


In [4]:
sentiment_structured_model = model.with_structured_output(Sentiment)
diagnosis_structured_model = model.with_structured_output(Diagnosis)

sentiment_template = PromptTemplate(
    template='Identify the sentiment of follwing review as positive or negative \nReview: {review}',
    input_variables=['review']
)

thanks_template = PromptTemplate(
    template='Write a warm thank you message in response to this review \nReview: {review}, \nAlso ask user to leave feedback and rate us',
    input_variables=['review']
)

diagnosis_template = PromptTemplate(
    template='Analyze the review and return issue_type, tone, sentiment_strength, specificity, constructiveness, authenticity, politeness, impact_severity, improvement_suggestion. \nReview: {review}',
    input_variables=['review']
)

apology_template = PromptTemplate(
    template= 'You are a customer support assistant; given the review "{review}" with issue_type={issue_type}, tone={tone}, sentiment_strength={sentiment_strength}, specificity={specificity}, constructiveness={constructiveness}, authenticity={authenticity}, politeness={politeness}, impact_severity={impact_severity}, and improvement_suggestion={improvement_suggestion}, write a short, empathetic, apology-themed response that acknowledges the problem and offers helpful next steps.',
    input_variables = ["review", "issue_type", "tone", "sentiment_strength", "specificity", "constructiveness", "authenticity", "politeness", "impact_severity", "improvement_suggestion"]
)

In [5]:
class Review_State(TypedDict):

    review: str
    sentiment: Literal['positive','negative']
    diagnosis:dict
    response:str

In [6]:
def find_sentiment(state:Review_State) -> Review_State:

    prompt = sentiment_template.invoke({'review':state['review']})
    sentiment = sentiment_structured_model.invoke(prompt).sentiment

    return {'sentiment':sentiment}


def run_diagnosis(state:Review_State) -> Review_State:
        
    prompt = diagnosis_template.invoke({'review':state['review']})
    diagnosis_dict = diagnosis_structured_model.invoke(prompt).model_dump()

    return {'diagnosis':diagnosis_dict}

def thanks_msg(state:Review_State) -> Review_State:

    prompt = thanks_template.invoke({'review':state['review']})
    response = model.invoke(prompt)

    return {'response':response.content}

def apology_msg(state:Review_State) -> Review_State:

    diagnosis = state['diagnosis']

    prompt = apology_template.invoke({
        "review": state["review"],
        "issue_type": diagnosis["issue_type"],
        "tone": diagnosis["tone"],
        "sentiment_strength": diagnosis["sentiment_strength"],
        "specificity": diagnosis["specificity"],
        "constructiveness": diagnosis["constructiveness"],
        "authenticity": diagnosis["authenticity"],
        "politeness": diagnosis["politeness"],
        "impact_severity": diagnosis["impact_severity"],
        "improvement_suggestion": diagnosis["improvement_suggestion"],
    })
    
    response = model.invoke(prompt)

    return {'response':response.content}


In [7]:
def conditional_router(state:Review_State) -> Literal['thanks_msg','run_diagnosis']:

    if state['sentiment'] == 'positive':
        return 'thanks_msg'
    else:
        return 'run_diagnosis'

In [8]:
graph = StateGraph(Review_State)

graph.add_node('find_sentiment',find_sentiment)
graph.add_node('run_diagnosis',run_diagnosis)
graph.add_node('thanks_msg',thanks_msg)
graph.add_node('apology_msg',apology_msg)

graph.add_edge(START,'find_sentiment')

graph.add_conditional_edges('find_sentiment',conditional_router)

graph.add_edge('thanks_msg',END)
graph.add_edge('run_diagnosis','apology_msg')
graph.add_edge('apology_msg',END)


workflow = graph.compile()

In [9]:
initial_state = {'review':'fascinating UI, loves it, but performance requires some fixes'}

final_state = workflow.invoke(initial_state)
final_state

{'review': 'fascinating UI, loves it, but performance requires some fixes',
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'product_quality',
  'tone': 'disappointed',
  'sentiment_strength': 'moderate',
  'specificity': 'somewhat_specific',
  'constructiveness': 'non_actionable',
  'authenticity': 'genuine',
  'politeness': 'polite',
  'impact_severity': 'medium',
  'improvement_suggestion': 'implicit'},
 'response': 'We’re sorry to hear that the performance isn’t living up to the great UI you love.\u202fWe understand how frustrating that can be.\u202fIf you could share a bit more detail—such as the device you’re using, the app version, and any specific actions that feel slow—we’ll be able to investigate quickly. In the meantime, please try updating to the latest version (if you haven’t already) and clearing the app cache, which often helps. Feel free to reach out to our support team at\u202fsupport@example.com\u202fwith the details, and we’ll work on a fix for you as soon as 